<a href="https://colab.research.google.com/github/mthanish/Sign-Link/blob/main/SignLink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREPROCESSING AND MODEL TRAINING

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import os

# --- Configuration ---
# Set all your paths and parameters here

# 1. Paths in Google Drive
DRIVE_ZIP_PATH = '/content/drive/MyDrive/SignLink 2.0/archive (1).zip'
MODEL_SAVE_DIR = '/content/drive/MyDrive/SignLink 2.0/MyModel'

# 2. Local Colab Paths (Temporary & Fast)
# We unzip the data here for fast I/O
LOCAL_DATA_DIR = '/content/dataset'

# 3. Model & Training Parameters
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 1

In [16]:
from google.colab import drive
import shutil

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Clean up old data and unzip to Colab's local disk
print(f"Preparing local directory: {LOCAL_DATA_DIR}")
# Remove old data if it exists, ensuring a fresh unzip
if os.path.exists(LOCAL_DATA_DIR):
    shutil.rmtree(LOCAL_DATA_DIR)
os.makedirs(LOCAL_DATA_DIR, exist_ok=True)

# Create the model save directory in your Drive
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print(f"Unzipping dataset from Drive to {LOCAL_DATA_DIR}...")
# Unzip quietly from your Drive to the fast local Colab disk
!unzip -q "{DRIVE_ZIP_PATH}" -d "{LOCAL_DATA_DIR}"

print("Unzipping complete. Data is ready.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Preparing local directory: /content/dataset
Unzipping dataset from Drive to /content/dataset...
Unzipping complete. Data is ready.


In [17]:
# --- 4. Load and Preprocess Data ---
DATA_DIR = "/content/dataset/images"

print(f"Loading data from: {DATA_DIR}")

# --- Load Training Data (80% of the images) ---
train_dataset = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# --- Load Validation Data (20% of the images) ---
validation_dataset = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Get class names and count
class_names = train_dataset.class_names
NUM_CLASSES = len(class_names)
print(f"\nFound {NUM_CLASSES} classes: {class_names}")

# --- Configure dataset for performance ---
AUTOTUNE = tf.data.AUTOTUNE

# CHANGED: We removed .cache() from both lines
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)



# --- ADD THIS CODE TO THE END OF CELL 3 ---

print("\n--- DEBUGGING CHECK ---")
try:
    train_batches = tf.data.experimental.cardinality(train_dataset)
    val_batches = tf.data.experimental.cardinality(validation_dataset)

    print(f"Number of batches in train_dataset: {train_batches.numpy()}")
    print(f"Number of batches in validation_dataset: {val_batches.numpy()}")

    if val_batches.numpy() == 0:
        print("\nERROR: Your validation dataset is EMPTY!")
        print("This is the reason for the 'KeyError'.")
        print("This might be because your dataset is too small or all files were assigned to 'train'.")
    else:
        print("\nSUCCESS: Your validation dataset is correctly loaded.")
except Exception as e:
    print(f"\nAn error occurred while checking datasets: {e}")

Loading data from: /content/dataset/images
Found 196621 files belonging to 110 classes.
Using 157297 files for training.
Found 196621 files belonging to 110 classes.
Using 39324 files for validation.

Found 110 classes: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'All_Gone', 'Alone', 'Baby', 'Beside', 'Book', 'Bowl', 'Bridge', 'Call', 'Camp', 'Cartridge', 'Flower', 'Fond', 'Glove', 'H', 'Hang', 'High', 'House', 'How_Many', 'I want Food', 'I_m Good', 'IorMe', 'J', 'Man', 'Marry', 'Meat', 'Medal', 'Mid_Day', 'Middle', 'Money', 'Moon', 'Mother', 'Opposite', 'Pain', 'Prisoner', 'Ring', 'Rose', 'See', 'Short', 'Stop', 'Superior', 'Theif', 'There is Gun', 'Thick', 'Thin', 'Tobacco', 'Up', 'V', 'Victory', 'Watch', 'Write', 'a', 'aboard', 'afraid', 'agree', 'assistance', 'b', 'bad', 'become', 'c', 'college', 'd', 'del', 'doctor', 'e', 'f', 'friend', 'from', 'g', 'i', 'k', 'l', 'm', 'n', 'not fine', 'nothing', 'o', 'ok fine', 'p', 'pray', 'q', 'r', 's', 'secondary', 'skin', 'small', 'spa

In [18]:
def build_model(input_shape, num_classes):
    """Creates and returns a new CNN model."""
    model = Sequential([
        Rescaling(1./255, input_shape=input_shape),

        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),

        Flatten(),

        Dense(256, activation='relu'),
        Dropout(0.5),

        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# --- Build the model ---
print("\nBuilding the CNN model...")
INPUT_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], 3)
model = build_model(INPUT_SHAPE, NUM_CLASSES)
model.summary()


Building the CNN model...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 110)            │        28,270 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,510,382 (32.46 MB)

 Trainable params: 8,510,382 (32.46 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# --- 7. Train the Model ---
import os
# Create a checkpoint to save the best model during training
# This is your safety net against Colab crashes
checkpoint_path = os.path.join(MODEL_SAVE_DIR, "best_model.h5")
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='val_accuracy',  # Save the model with the best validation accuracy
    mode='max',
    save_best_only=True,     # Only save if it's better than the previous best
    verbose=1
)

print("\n--- Starting Model Training ---")

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]  # Pass in the checkpoint callback
)

print("\n--- Training Complete ---")
print(f"The best model was saved to {checkpoint_path}")


--- Starting Model Training ---
4916/4916 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.3692 - loss: 2.3498
Epoch 1: val_accuracy improved from -inf to 0.90070, saving model to /content/drive/MyDrive/SignLink 2.0/MyModel/best_model.h5


4916/4916 ━━━━━━━━━━━━━━━━━━━━ 198s 40ms/step - accuracy: 0.3693 - loss: 2.3496 - val_accuracy: 0.9007 - val_loss: 0.3590

--- Training Complete ---
The best model was saved to /content/drive/MyDrive/SignLink 2.0/MyModel/best_model.h5


In [21]:
# --- 7. Train the Model (AND Plot) ---
import os
import matplotlib.pyplot as plt

# --- Checkpoint Setup ---
checkpoint_path = os.path.join(MODEL_SAVE_DIR, "best_model.h5")
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# --- Define the Plotting Function ---
def plot_training_history(history):
    """Plots the accuracy and loss from the training history."""

    # Use .get() to safely check for keys. This avoids the KeyError.
    acc = history.history.get('accuracy', [])
    val_acc = history.history.get('validation_accuracy', [])
    loss = history.history.get('loss', [])
    val_loss = history.history.get('validation_loss', [])

    if not acc or not val_acc:
        print("\n--- PLOTTING ERROR ---")
        print("Could not find 'accuracy' or 'validation_accuracy' in history.")
        print("This means validation did not run. Check your 'validation_data' argument.")
        print(f"Keys found: {history.history.keys()}")
        return

    epochs_range = range(len(acc))
    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

# --- Start Training ---
print("\n--- Starting Model Training (for 1 Epoch) ---")

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS,  # This should be 1 from Cell 1
    callbacks=[checkpoint_callback]
)

print("\n--- Training Complete ---")

# --- Plot Results Immediately ---
print("\n--- Plotting Results ---")
plot_training_history(history)

# --- Save the Final Epoch Model ---
final_model_path = os.path.join(MODEL_SAVE_DIR, 'final_epoch_model.h5')
model.save(final_model_path)
print(f"Final epoch model saved to {final_model_path}")


--- Starting Model Training (for 1 Epoch) ---
4916/4916 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7965 - loss: 0.6219
Epoch 1: val_accuracy improved from -inf to 0.95072, saving model to /content/drive/MyDrive/SignLink 2.0/MyModel/best_model.h5


4916/4916 ━━━━━━━━━━━━━━━━━━━━ 179s 36ms/step - accuracy: 0.7965 - loss: 0.6219 - val_accuracy: 0.9507 - val_loss: 0.1698



--- Training Complete ---

--- Plotting Results ---

--- PLOTTING ERROR ---
Could not find 'accuracy' or 'validation_accuracy' in history.
This means validation did not run. Check your 'validation_data' argument.
Keys found: dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])
Final epoch model saved to /content/drive/MyDrive/SignLink 2.0/MyModel/final_epoch_model.h5


In [22]:
import json
import os

# --- This assumes your 'class_names' variable is still in memory ---
# If it's not, just re-run Cell 3 to create it again

if 'class_names' in locals():
    # Get the MODEL_SAVE_DIR from Cell 1
    labels_path = os.path.join(MODEL_SAVE_DIR, 'labels.json')

    with open(labels_path, 'w') as f:
        json.dump(class_names, f)

    print(f"Labels saved successfully to: {labels_path}")
    print("You can now download 'best_model.h5' and 'labels.json'")
else:
    print("Error: 'class_names' not found. Please re-run Cell 3 to generate it.")



Labels saved successfully to: /content/drive/MyDrive/SignLink 2.0/MyModel/labels.json
You can now download 'best_model.h5' and 'labels.json'
